# IMDB movie reviews binary classification

## Setup

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_name = tf.test.gpu_device_name()
device_name

2024-01-25 17:34:51.458910: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 17:34:51.458986: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-25 17:34:51.459027: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Num GPUs Available:  1


'/device:GPU:0'

In [5]:
# A dependency of the preprocessing for BERT inputs
!pip install -qU tensorflow-text==2.14.0

In [6]:
!pip install -qU tf-models-official==2.14.0

In [7]:
!python --version

Python 3.10.12


In [8]:
import os
import shutil
import os.path as path

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [9]:
tf.__version__

'2.14.1'

## Dataset

In [10]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

if not path.exists("/content/aclImdb"):
  dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

  dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

  train_dir = os.path.join(dataset_dir, 'train')

  # remove unused folders to make it easier to load the data
  remove_dir = os.path.join(train_dir, 'unsup')
  shutil.rmtree(remove_dir)

In [11]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 1
validation_split = 0.1

# enables operations determinism
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=validation_split,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names

train_ds = raw_train_ds.cache()

val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=validation_split,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache()

test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache()

Found 25000 files belonging to 2 classes.
Using 22500 files for training.
Found 25000 files belonging to 2 classes.
Using 2500 files for validation.
Found 25000 files belonging to 2 classes.


In [12]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print(f'Review: {text_batch.numpy()[i]}')
    label = label_batch.numpy()[i]
    print(f'Label : {label} ({class_names[label]})')

Review: b"I entered the theatre intending to pass a pleasant 90 minutes being entertained if not enlightened. I left neither entertained nor enlightened. This movie can't make up its mind what it wants to be and ends up being not much of anything. There are a few funny lines and a few incredibly pretentious movie references (The 400 Blows--for this character? come off it!). While none of the characters gets treated with much respect, the over thirty gay men get the worst of it: all predatory, fat, sad, slobs. If you're in the mood for a movie dealing with gay relationships check out Parting Glances, Longtime Companion, Trick, All Over the Guy, Red Dirt, Maurice, Philadelphia instead. You'll thank me.<br /><br />"
Label : 0 (neg)
Review: b"Though I liked On the Town better I really liked it. I'm a new comer when it comes to Frank Sinatra and Gene Kelly. Though I had heard of them I had never seen anything with them in it until recently. The first one I saw was Singin in the Rain that ma

## Vocabolary

## PRE-trained Model

In [15]:
url = 'https://github.com/giuliocn/bert_binary_classification_imdb_reviews/releases/download/binary-classification/imdb_l10_h256_a4_bert-20231126T160856Z-001.zip'
if not path.exists("imdb_l10_h256_a4_bert"):
  model = tf.keras.utils.get_file(
      'imdb_l10_h256_a4_bert-20231126T160856Z-001.zip',
      url,
      extract=True, cache_dir='.',
      cache_subdir='')

In [16]:
# Load model from directory
classifier_model = tf.keras.models.load_model('imdb_l10_h256_a4_bert')

# Check model architecture
classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(32,)]                      0         []                            
                                                                                                  
 pre_layer (Custom>PreLayer  {'input_type_ids': (None,    0         ['input_1[0][0]']             
 )                           256),                                                                
                              'input_word_ids': (None,                                            
                             256),                                                                
                              'input_mask': (None, 256)                                           
                             }                                                                

## Craft Vocabolary

In [18]:
test_ds

<CacheDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [21]:
output = classifier_model(tf.constant(["This movie was very good! Excellent."]*batch_size))
output = tf.math.reduce_mean(output)
output

<tf.Tensor: shape=(), dtype=float32, numpy=0.97050166>

## Evaluation

In [22]:
# Evaluate the model on metrics
classifier_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    name='binary_crossentropy'
    ),
    metrics=[tf.keras.metrics.BinaryAccuracy()])

In [24]:
result = classifier_model.evaluate(
    test_ds,
    return_dict=True)

782/782 [==============================] - 211s 266ms/step - loss: 0.4768 - binary_accuracy: 0.8397


In [27]:
result

{'loss': 0.4767884314060211, 'binary_accuracy': 0.8397200107574463}

In [26]:
string = f"""
{'Precision':15}  {result['precision']:.3f}
{'Recall':15}  {result['recall']:.3f}

{'True Positives':15}  {int(result['true_positives'])*100/25e3:2.3f} %
{'False Negatives':15}  {int(result['false_negatives'])*100/25e3:2.3f} %
{'False Positives':15}  {int(result['false_positives'])*100/25e3:2.3f} %
{'True Negatives':15}  {int(result['true_negatives'])*100/25e3:2.3f} %
"""
print(string)

KeyError: 'precision'

## Unseen Examples

In [28]:
examples = []

for tf_text, tf_label in test_ds.unbatch().shuffle(1).take(5):
  examples.append(tf_text.numpy())
  print(tf_text.numpy()[:50], f"Expected output: {tf_label.numpy()}")
  result = classifier_model(tf.constant([tf_text.numpy()] * batch_size))
  result = tf.math.reduce_mean(result)
  print(f"model result: {result.numpy():.3f}")


b'This is a tough film to review, since several fact' Expected output: 1
model result: 0.007
b'This is a charming little film, which like many of' Expected output: 1
model result: 0.976
b'I remember viewing this movie when I was a kid. I ' Expected output: 0
model result: 0.959
b'This odd little film starts out with the story of ' Expected output: 0
model result: 0.989
b"I have seen a few of Fred Carpenter's movies on Sh" Expected output: 1
model result: 0.999


## More Examples

In [29]:
result = classifier_model(tf.constant(['The movie was great!'] * batch_size))
result = tf.math.reduce_mean(result)
print(f"model result: {result.numpy():.3f}")

model result: 0.947


In [30]:
# Fails to read emojis
result = classifier_model(tf.constant(['👍'] * batch_size))
result = tf.math.reduce_mean(result)
print(f"model result: {result.numpy():.3f}")

model result: 0.078


In [31]:
result = classifier_model(tf.constant(['The movie was terrible!'] * batch_size))
result = tf.math.reduce_mean(result)
print(f"model result: {result.numpy():.3f}")

model result: 0.163


In [32]:
# Model fails to classify single positive words
result = classifier_model(tf.constant(['Wonderful.'] * batch_size))
result = tf.math.reduce_mean(result)
print(f"model result: {result.numpy():.3f}")

model result: 0.128


In [33]:
result = classifier_model(tf.constant(['Terrible.'] * batch_size))
result = tf.math.reduce_mean(result)
print(f"model result: {result.numpy():.3f}")

model result: 0.136


In [34]:
# Neutral statement
result = classifier_model(tf.constant(['This is a movie'] * batch_size))
result = tf.math.reduce_mean(result)
print(f"model result: {result.numpy():.3f}")

model result: 0.491


## Classify single words with a template string

In [35]:
good_words = [
    # Good
    'good','great','enjoyable','amazing','delightful','lovely','pleasant',]
bad_words = [
    # Bad
    'bad','poor','inferior','lacking','awful','terrible','abominable',]
neutral_words = [
    # Neutral
    'indifferent','mediocre','ordinary','average','commonplace','medium','moderate',
]

def print_my_examples(words):
  for w in words:
    result = classifier_model(tf.constant([f"This movie was {w}"] * 32))
    result = tf.math.reduce_mean(result)
    print(f"input: {w:15}\t\t result:{result:.3f}", sep='\n')


In [36]:
print_my_examples(good_words)
print('\n')
print_my_examples(bad_words)
print('\n')
print_my_examples(neutral_words)

input: good           		 result:0.624
input: great          		 result:0.783
input: enjoyable      		 result:0.832
input: amazing        		 result:0.842
input: delightful     		 result:0.877
input: lovely         		 result:0.692
input: pleasant       		 result:0.828


input: bad            		 result:0.090
input: poor           		 result:0.107
input: inferior       		 result:0.066
input: lacking        		 result:0.074
input: awful          		 result:0.074
input: terrible       		 result:0.077
input: abominable     		 result:0.060


input: indifferent    		 result:0.157
input: mediocre       		 result:0.145
input: ordinary       		 result:0.459
input: average        		 result:0.231
input: commonplace    		 result:0.843
input: medium         		 result:0.067
input: moderate       		 result:0.425
